This is the jupyter file to be used for running each part of the code. Hover over the cell and click the run button to run the code. The code is split into 3 parts: Image Processing, Matching, and Assembling. Each part is run separately and the results are displayed in the output.

The original author of this application is Maxim Terleev. The source files can be seen in the following GitHub repository:
https://github.com/MaximTerleev/Jigsaw-Puzzle-AI/blob/main/jigsaw-puzzle-solver.py

The solver contains many functions and can be split into 3 main sections:
1. Image Processing
2. Matching
3. Assembling

In [ ]:
# imports

from scipy.ndimage import filters
from PIL import Image, ImageChops
import matplotlib.pyplot as plt
from fastdtw import fastdtw
import numpy as np
import cv2